In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import ast

In [ ]:

dataset_path = "/home/jmzzz/Repos/topicosnlp/RBM/data.csv"  

# Cargar el dataset con pandas
df = pd.read_csv(dataset_path)

# Procesar la columna 'piano_roll' convirtiendo el string en un array NumPy
piano_rolls = []
for index, row in df.iterrows():
    piano_roll_str = row["piano_roll"]
    try:
        # Es posible que el string incluya saltos de línea; eliminarlos y evaluar el string
        piano_roll_str = piano_roll_str.replace('\n', ' ').strip()
        piano_roll_array = np.array(ast.literal_eval(piano_roll_str))
    except Exception as e:
        print(f"Error al convertir 'piano_roll' en la fila {index}: {e}")
        continue
    piano_rolls.append(piano_roll_array)

if len(piano_rolls) == 0:
    raise ValueError("No se pudo cargar ningún piano roll desde el dataset.")

# Asumimos que todos los piano rolls tienen la misma forma, por ejemplo, (88, time_steps)
piano_roll_shape = piano_rolls[0].shape
print("Forma de cada piano_roll:", piano_roll_shape)

# Convertir la lista de arrays a un solo array de NumPy y aplanarlo:
data_np = np.stack(piano_rolls, axis=0)       # Forma: (n_samples, 88, time_steps)
n_samples = data_np.shape[0]
data_np_flat = data_np.reshape(n_samples, -1)   # Cada ejemplo es un vector

# Convertir a tensor de PyTorch
data_tensor = torch.tensor(data_np_flat, dtype=torch.float)

# ================================================
# 2. Definir las clases RBM y DBN
# ================================================
class RBM(nn.Module):
    def __init__(self, n_visible, n_hidden):
        super(RBM, self).__init__()
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        # Inicializar pesos y biases
        self.W = nn.Parameter(torch.randn(n_hidden, n_visible) * 0.1)
        self.h_bias = nn.Parameter(torch.zeros(n_hidden))
        self.v_bias = nn.Parameter(torch.zeros(n_visible))
    
    def sample_h(self, v):
        # Calcula la probabilidad de activación de las unidades ocultas
        p_h = torch.sigmoid(torch.matmul(v, self.W.t()) + self.h_bias)
        h_sample = torch.bernoulli(p_h)
        return p_h, h_sample
    
    def sample_v(self, h):
        # Calcula la probabilidad de activación de las unidades visibles
        p_v = torch.sigmoid(torch.matmul(h, self.W) + self.v_bias)
        v_sample = torch.bernoulli(p_v)
        return p_v, v_sample
    
    def forward(self, v):
        # Paso de codificación y reconstrucción
        _, h = self.sample_h(v)
        _, v_reconstructed = self.sample_v(h)
        return v_reconstructed

    def contrastive_divergence(self, v, k=1, lr=0.01):
        v0 = v
        for _ in range(k):
            p_h, h = self.sample_h(v)
            p_v, v = self.sample_v(h)
        # Cálculo de gradientes: diferencia entre la activación de la capa oculta en la muestra original y la reconstruida
        p_h0, _ = self.sample_h(v0)
        p_hk, _ = self.sample_h(v)
        # Actualización manual de parámetros
        self.W.grad = -(torch.matmul(p_h0.t(), v0) - torch.matmul(p_hk.t(), v))
        self.v_bias.grad = -torch.sum(v0 - v, dim=0)
        self.h_bias.grad = -torch.sum(p_h0 - p_hk, dim=0)
        for param in [self.W, self.v_bias, self.h_bias]:
            param.data -= lr * param.grad
        loss = torch.mean(torch.sum((v0 - v) ** 2, dim=1))
        return loss

class DBN:
    def __init__(self, layers, lr=0.01, k=1, device='cpu'):
        """
        layers: lista con los tamaños de cada capa [n_visible, n_hidden1, n_hidden2, ...]
        """
        self.device = device
        self.rbms = []
        self.lr = lr
        self.k = k
        for i in range(len(layers) - 1):
            rbm = RBM(layers[i], layers[i+1]).to(device)
            self.rbms.append(rbm)
    
    def train(self, data, epochs=10, batch_size=64):
        num_samples = data.size(0)
        for idx, rbm in enumerate(self.rbms):
            print(f"Entrenando RBM {idx+1}/{len(self.rbms)}")
            for epoch in range(epochs):
                epoch_loss = 0
                permutation = torch.randperm(num_samples)
                for i in range(0, num_samples, batch_size):
                    batch_idx = permutation[i:i+batch_size]
                    batch = data[batch_idx].to(self.device)
                    loss = rbm.contrastive_divergence(batch, k=self.k, lr=self.lr)
                    epoch_loss += loss.item()
                print(f"  Época {epoch+1}: Loss = {epoch_loss/num_samples:.4f}")
            # Transformar los datos para la siguiente RBM
            _, data = rbm.sample_h(data.to(self.device))
    
    def generate(self, seed, steps=1000):
        # Elevar la semilla a la representación de la última capa
        v = seed.to(self.device)
        for rbm in self.rbms[:-1]:
            _, v = rbm.sample_h(v)
        # En la capa superior, realizar Gibbs sampling
        top_rbm = self.rbms[-1]
        for _ in range(steps):
            _, h = top_rbm.sample_h(v)
            _, v = top_rbm.sample_v(h)
        # Reconstrucción descendente
        for rbm in reversed(self.rbms[:-1]):
            _, v = rbm.sample_v(v)
        return v

    def save(self, path):
        # Guarda el estado de cada RBM
        state = {'rbms': [rbm.state_dict() for rbm in self.rbms]}
        torch.save(state, path)
        print(f"Modelo guardado en {path}")
    
    def load(self, path):
        state = torch.load(path)
        for rbm, rbm_state in zip(self.rbms, state['rbms']):
            rbm.load_state_dict(rbm_state)
        print(f"Modelo cargado desde {path}")

# ================================================
# 3. Configuración y Entrenamiento
# ================================================
# Usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# Definir la arquitectura de la DBN
input_size = data_tensor.shape[1]  # Tamaño del piano roll aplanado
# Ejemplo: dos capas ocultas con 500 y 200 neuronas
layers = [input_size, 512, 256,128]

# Crear e instanciar la DBN
dbn = DBN(layers, lr=0.01, k=1, device=device)

# Entrenar la DBN
print("Iniciando entrenamiento de la DBN...")
dbn.train(data_tensor, epochs=10, batch_size=16)

# ================================================
# 4. Generación y Guardado del Modelo
# ================================================
# Usar una semilla: por ejemplo, el primer piano roll del dataset
seed = data_tensor[0]
generated = dbn.generate(seed, steps=500)

# Reconstruir la forma original del piano roll
generated_np = generated.cpu().detach().numpy().reshape(piano_roll_shape)
print("Piano roll generado (formato NumPy):")
print(generated_np)

# Guardar el modelo entrenado
dbn.save("modelo_dbn_musica.pth")


In [9]:
import pandas as pd
import numpy as np
import ast
import torch

# Ruta del CSV
csv_path = "/home/jmzzz/Repos/topicosnlp/RBM/data.csv"

# Cargar el dataset
df = pd.read_csv(csv_path)

# Lista para almacenar los piano rolls convertidos
piano_rolls = []

# Recorrer cada fila del DataFrame y convertir la columna 'piano_roll'
for index, row in df.iterrows():
    piano_roll_str = row["piano_roll"]
    
    # Si encuentras "..." significa que la representación está truncada.
    if "..." in piano_roll_str:
        print(f"Fila {index}: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.")
        continue

    try:
        # Convertir la cadena a un array de Python y luego a un array NumPy
        piano_roll_array = np.array(ast.literal_eval(piano_roll_str))
    except Exception as e:
        print(f"Error al convertir 'piano_roll' en la fila {index}: {e}")
        continue

    piano_rolls.append(piano_roll_array)

# Verificar que se cargó al menos un piano roll completo
if len(piano_rolls) == 0:
    raise ValueError("No se pudo cargar ningún piano roll completo desde el dataset.")

# Asumir que todos los piano rolls tienen la misma forma
piano_roll_shape = piano_rolls[0].shape
print("Forma de cada piano roll:", piano_roll_shape)

# Convertir la lista de arrays a un solo array de NumPy y aplanarlo
data_np = np.stack(piano_rolls, axis=0)       # Forma: (n_samples, height, width)
n_samples = data_np.shape[0]
data_np_flat = data_np.reshape(n_samples, -1)   # Cada ejemplo es un vector

# Convertir a tensor de PyTorch
data_tensor = torch.tensor(data_np_flat, dtype=torch.float)

# Ahora ya tienes 'data_tensor' listo para usar en tu DBN.
print("Dataset cargado correctamente. Tensor shape:", data_tensor.shape)


Fila 0: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 1: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 2: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 3: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 4: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 5: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 6: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 7: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 8: Se encontró '...' en la representación. Asegúrate de que el piano roll esté guardado completo.
Fila 9: Se encontró '...' en la representación. Asegúrate de que el piano

ValueError: No se pudo cargar ningún piano roll completo desde el dataset.